In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_groq import ChatGroq
groq_api_key = os.getenv("GRQO_API_KEY")
llm = ChatGroq(groq_api_key= groq_api_key, model="Llamma3.2-8b-8192")



In [ ]:
from langchain.schema import AIMessage, HumanMessage, SystemMessage



In [ ]:
speech = """ The speech of Imran khan ex Pakistani prime minister in United nations is one of the famous speech of Pkaistani Prime minister.
He spot light on four different but ignored topics like Kashmir issue, Islam od Phobia, Climate Change, Palestine Issue. 

"""